#### This notebook explains how to uncover risks related to your usecase based on a given taxonomy.


##### Import libraries


In [1]:
from ai_atlas_nexus.blocks.inference import (
    RITSInferenceEngine,
    WMLInferenceEngine,
    OllamaInferenceEngine,
    VLLMInferenceEngine,
)
from ai_atlas_nexus.blocks.inference.params import (
    InferenceEngineCredentials,
    RITSInferenceEngineParams,
    WMLInferenceEngineParams,
    OllamaInferenceEngineParams,
    VLLMInferenceEngineParams,
)
from ai_atlas_nexus.library import AIAtlasNexus

/Users/dhaval/Projects/Usage-Governance/ai-atlas-nexus/src/ai_atlas_nexus/blocks/risk_explorer/explorer.py:8: UserWarning: Deprecated, Please use AtlasExplorer instead
  warnings.warn("Deprecated, Please use AtlasExplorer instead")
/Users/dhaval/Projects/Usage-Governance/ai-atlas-nexus/src/ai_atlas_nexus/toolkit/job_utils.py:4: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm


##### AI Atlas Nexus uses Large Language Models (LLMs) to infer risks dimensions. Therefore requires access to LLMs to inference or call the model.

**Available Inference Engines**: WML, Ollama, vLLM, RITS. Please follow the [Inference APIs](https://github.com/IBM/ai-atlas-nexus?tab=readme-ov-file#install-for-inference-apis) guide before going ahead.

_Note:_ RITS is intended solely for internal IBM use and requires TUNNELALL VPN for access.


In [2]:
# inference_engine = OllamaInferenceEngine(
#     model_name_or_path="granite3.3:8b",
#     credentials=InferenceEngineCredentials(api_url="OLLAMA_API_URL"),
#     parameters=OllamaInferenceEngineParams(
#         num_predict=1000, num_ctx=8192, temperature=0, repeat_penalty=1
#     ),
# )

# inference_engine = WMLInferenceEngine(
#     model_name_or_path="ibm/granite-3-3-8b-instruct",
#     credentials={
#         "api_key": "WML_API_KEY",
#         "api_url": "WML_API_URL",
#         "project_id": "WML_PROJECT_ID",
#     },
#     parameters=WMLInferenceEngineParams(
#         max_new_tokens=1000, decoding_method="greedy", repetition_penalty=1
#     ),
# )

# inference_engine = VLLMInferenceEngine(
#     model_name_or_path="ibm-granite/granite-3.3-8b-instruct",
#     credentials=InferenceEngineCredentials(
#         api_url="VLLM_API_URL", api_key="VLLM_API_KEY"
#     ),
#     parameters=VLLMInferenceEngineParams(max_tokens=1000, temperature=0),
# )

inference_engine = RITSInferenceEngine(
    model_name_or_path="ibm-granite/granite-3.3-8b-instruct",
    credentials={
        "api_key": "cbc683b3a1a7c52d2a73008b785d2811",
        "api_url": "https://inference-3scale-apicast-production.apps.rits.fmaas.res.ibm.com",
    },
    parameters=RITSInferenceEngineParams(
        max_completion_tokens=1000, temperature=0, logprobs=True, top_logprobs=3
    ),
)

[2025-12-16 14:23:43:130] - INFO - AIAtlasNexus - Created RITS inference engine.


##### Create an instance of AIAtlasNexus

_Note: (Optional)_ You can specify your own directory in `AIAtlasNexus(base_dir=<PATH>)` to utilize custom AI ontologies. If left blank, the system will use the provided AI ontologies.


In [3]:
ai_atlas_nexus = AIAtlasNexus()

[2025-12-16 14:23:43:383] - INFO - AIAtlasNexus - Created AIAtlasNexus instance. Base_dir: None


##### Risk Identification API

AIAtlasNexus.identify_risks_from_usecases()

Params:

- usecases (List[str]):
  A List of strings describing AI usecases
- inference_engine (InferenceEngine):
  An LLM inference engine to infer risks from the usecases.
- taxonomy (str, optional):
  The string label for a taxonomy. Default to None.
- cot_examples (Dict[str, List], optional):
  The Chain of Thought (CoT) examples to use in the risk identification.
  The example template is available at src/ai_atlas_nexus/data/templates/risk_generation_cot.json.
  Assign the ID of the taxonomy you wish to use as the key for CoT examples. Providing this value
  will override the CoT examples present in the template master. Default to None.
- max_risk (int, optional):
  The maximum number of risks to extract. Pass None to allow the inference engine to determine the number of risks. Defaults to None.
- zero_shot_only (bool): If enabled, this flag allows the system to perform Zero Shot Risk identification, and the field `cot_examples` will be ignored.


#### Usecase

In [4]:
usecase = "Using AI to write a government policy report on automated decision-making in Ireland's welfare system. The task will include  drafting a comprehensive section analyzing input documents to understand how Ireland federal courts have addressed automated penalties in welfare systems and ensure the output is factually correct."

##### Risk Identification using IBM AI Risk taxonomy - Batch Inference


In [ ]:
risks = ai_atlas_nexus.identify_risks_from_usecases(
    usecases=[usecase],
    inference_engine=inference_engine,
    taxonomy="ibm-risk-atlas",
    max_risk=5,
    batch_inference=True,
)

print("No. of risks:", len(risks[0]))
for risk in risks[0]:
    print(risk.name)

Inferring with RITS:   0%|          | 0/1 [00:00<?, ?it/s]

No. of risks: 5
Incorrect risk testing
Over- or under-reliance
Discriminatory actions
Harmful output
Lack of model transparency


##### Risk Identification using IBM AI Risk taxonomy - Inference per risk level


In [5]:
import json

risks = ai_atlas_nexus.identify_risks_from_usecases(
    usecases=[usecase],
    inference_engine=inference_engine,
    taxonomy="ibm-risk-atlas",
    batch_inference=False,
)

print("No. of risks:", len(risks[0]))
for risk in risks[0]:
    print(risk.name)

Inferring with RITS:   0%|          | 0/99 [00:00<?, ?it/s]

No. of risks: 2
Hallucination
Impact on affected communities
